In [ ]:
import IJulia
IJulia.installkernel("Bilevel_DNEP", "--project=$(Base.active_project())")

In [1]:
include("main.jl")

#--------------------------------------------------------------------------------
#                                                                                
#                                  Bilevel DNEP                                   
#                                                                                
#                                Graduation work                                 
#                                                                                
#--------------------------------------------------------------------------------
# @ Manon Cornet

Running a simulation with the following characteristics:
--------------------------------------------------------

╭─────────────┬───────┬───────┬─────────┬─────────┬─────────┬─────────┬─────────┬───────────╮
│  Model Type │    EV │   EHP │ Delta_t │ PV_CAPA │     EIC │     EEC │   DSOEC │       GCC │
│         [-] │   [-] │   [-] │   [min] │   [MVA] │ [€/kWh] │ [€/kWh] │ [€/kWh] │ [€/kVA/y] │
├─────────

[ Info: Building model...
[ Info: Built model in 4.95 seconds


Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-10
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 1
Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 1
Set parameter TimeLimit to value 600
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 633 rows, 475 columns and 2298 nonzeros
Model fingerprint: 0xd1e9f974
Model has 36 quadratic constraints
Variable types: 371 continuous, 104 integer (104 binary)
Coefficient statistics:
  Matrix range     [3e-08, 1e+02]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [2e-02, 2e+01]
  Bounds range     [9e-01, 1e+00]
  RHS range        [2e-03, 2e+01]
Presolve removed 4 rows and 4 columns
Presolve time: 0.00s
Presolved: 629 rows, 471 columns, 2288 nonzeros
Presolved model has 36 quadratic constraint(s

(12.989483797624773, 0.942432165145874)

In [ ]:
function communicate(cmd::Cmd, input)
    inp = Pipe()
    out = Pipe()
    err = Pipe()

    process = run(pipeline(cmd, stdin=inp, stdout=out, stderr=err), wait=false)
    close(out.in)
    close(err.in)

    stdout = @async String(read(out))
    stderr = @async String(read(err))
    write(process, input)
    close(inp)
    wait(process)
    return (
        stdout = fetch(stdout),
        stderr = fetch(stderr),
        code = process.exitcode
    )
end


a, = communicate(`julia --project ./main.jl`, "output.txt"); println(a)


In [ ]:
print(network.lines)

In [ ]:
using JuMP
latex_formulation(upper_model)
# Problem is that etiher P_ij or Q_ij_k

In [ ]:
objective, time = UpperLevel.solve_model(upper_model, formulation.powerflow)
save_struct(network_topology, "network_topology.json")
save_struct(network, "network_data.json")
# 1. What to do is to update the structures when going through this function
# 2. 


In [2]:
# --- https://tikz.dev/tikz-graphs#sec-19.7 ---
"""Draws a graph using TikZ."""
function showgraph_tikz(network, time_step, x_scale, y_scale;
                   filename="./graph",
                   display=true)

    # Let's insert some boilerplate styling
    # and necessary preamble/postamble
    preamble = """\\documentclass{standalone}
    \\usepackage{tikz}
    \\usepackage{amsmath}
    \\usetikzlibrary{graphs, quotes, arrows.meta, positioning}

    \\begin{document}
    \\begin{tikzpicture}[
        every label/.style = {align=center, font=\\tiny, inner sep=2pt},
        every edge quotes/.style = {font=\\scriptsize, text=black, fill=white, inner sep=2pt}
        ]
    \\graph [no placement]
    {\n"""

    style_sub = "rectangle, draw, fill=white, minimum size=1em, inner sep=1pt"
    style_load = "circle, draw, fill=white, minimum size=1em, inner sep=1pt"
    postamble = """};
    \\end{tikzpicture}
    \\end{document}"""
 

    base_power = network.pu_basis.base_power
    open(filename * ".tex", "w") do file
        write(file, preamble)
        
        for b in network.buses
            node = b.node
            n = node.id
            x = x_scale * node.coord.x
            y = y_scale * node.coord.y
            if n <= get_nb_substations(network)
                write(file,
                "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{S}_{$n}\$}, $(style_sub)];\n")
            else 
                write(file,
                "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{L}_{$n}\$}, $(style_load)];\n")
            end
        end

        for l in network.lines
            if l.built
                if l.P_send[time_step] >= 0
                    i = l.edge.from_node.id ; j = l.edge.to_node.id
                    p = l.P_send[time_step] * base_power
                else 
                    i = l.edge.to_node.id ; j = l.edge.from_node.id
                    p = abs(l.P_rec[time_step]) * base_power
                end
                write(file,
                "    $(i) ->[thick, \"$(round(p, digits=3))\"] $(j);\n")
            else 
                i = l.edge.from_node.id ; j = l.edge.to_node.id
                write(file,
                "    $(i) --[gray, dashed] $(j);\n")
            end       
        end  
        write(file, postamble)
    end
    

    run(pipeline(`lualatex $filename`))
    if display
        run(`open $filename.pdf`, wait=false)
    end
end

showgraph_tikz

In [ ]:
# --- https://tikz.dev/tikz-graphs#sec-19.7 ---
"""Draws a graph using TikZ."""
function showgraph_tikz(network, time_step, x_scale, y_scale;
                   filename="./graph",
                   display=true)

    # Let's insert some boilerplate styling
    # and necessary preamble/postamble
    preamble = """\\documentclass{standalone}
    \\usepackage{tikz}
    \\usetikzlibrary{graphs, quotes, positioning, arrows.meta}

    \\begin{document}
    \begin{tikzpicture}[
    house/.style={
        draw, minimum width=8mm, minimum height=6mm, 
        label={[name=labelaux, inner sep=1mm, distance=0pt]#1}, 
        append after command={%
            \pgfextra \draw ([xshift=.5\pgflinewidth]\tikzlastnode.north west)--(labelaux.north)
                --([xshift=-.5\pgflinewidth]\tikzlastnode.north east);\endpgfextra
        }
    },
    vertex/.style={draw,circle,minimum size=0.8cm},
    ]
    \\begin{tikzpicture}[substation/.style={draw,rectangle}, load/.style={draw,rectangle}]
    \\graph [no placement, nodes={draw,circle}, edge quotes={fill=white, inner sep=1pt}]
    {\n"""

    postamble = """};
    \\end{tikzpicture}
    \\end{document}"""

    base_power = network.pu_basis.base_power
    open(filename * ".tex", "w") do file
        write(file, preamble)
        
        for i in 1:network.nb_substations
            b = network.buses[i]
            node = b.node
            n = node.id
            x = x_scale * node.coord.x
            y = y_scale * node.coord.y
            style = "fill=white, text=black"

            write(file,
                  "    $n [x=$(x)cm, y=$(y)cm, as={\$n_{$n}\$}, $(style)];\n")
        end

        for l in network.lines
            if l.built
                if l.P_send[time_step] >= 0
                    i = l.edge.from_node.id ; j = l.edge.to_node.id
                    p = l.P_send[time_step] * base_power
                else 
                    i = l.edge.to_node.id ; j = l.edge.from_node.id
                    p = abs(l.P_rec[time_step]) * base_power
                end
                write(file,
                "    $(i) ->[thick, \"$(round(p, digits=3))\"] $(j);")
            else 
                i = l.edge.from_node.id ; j = l.edge.to_node.id
                write(file,
                "    $(i) --[gray, dashed] $(j);")
            end
            
            # 
            # P_sending = l.P_send >= 0 ? l.P_send : abs(l.P_rec) 
            # l.P_rec         
        end
        write(file, postamble)
    end

    run(pipeline(`lualatex $filename`))
    if display
        run(`open $filename.pdf`, wait=false)
    end
end

In [4]:
# -- Creation of the result plot --
using Graphs

g = Graph(get_nb_nodes(network_topology))
for l in network.lines 
    e = l.edge
    add_edge!(g, e.from_node.id, e.to_node.id)
end


showgraph_tikz(network, 1, 4, 4)


This is LuaHBTeX, Version 1.15.0 (TeX Live 2022) 
 restricted system commands enabled.
(./graph.tex
LaTeX2e <2021-11-15> patch level 1
 L3 programming layer <2022-02-24>
(/usr/local/texlive/2022/texmf-dist/tex/latex/standalone/standalone.cls
Document Class: standalone 2018/03/26 v1.3a Class to compile TeX sub-files stan
dalone
(/usr/local/texlive/2022/texmf-dist/tex/latex/tools/shellesc.sty)
(/usr/local/texlive/2022/texmf-dist/tex/generic/iftex/ifluatex.sty
(/usr/local/texlive/2022/texmf-dist/tex/generic/iftex/iftex.sty))
(/usr/local/texlive/2022/texmf-dist/tex/latex/xkeyval/xkeyval.sty
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/xkeyval.tex
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/xkvutils.tex
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/keyval.tex))))
(/usr/local/texlive/2022/texmf-dist/tex/latex/standalone/standalone.cfg)
(/usr/local/texlive/2022/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX docume

Process(`open ./graph.pdf`, ProcessRunning)

In [ ]:
using Graphs
using GraphViz
G = Graph(3) # graph with 3 vertices

# make a triangle
add_edge!(G, 1, 2)
add_edge!(G, 1, 3)
add_edge!(G, 2, 3)

for e in edges(G)
    print(e)
end
# print(nv(G))
# W=zeros(Float64, nv(G), nv(G))
# print(W[1,1])
# for i in 1:nv(G)
#     for j in 1:nv(G)
#         if has_edge(G, i, j)
#             print(i,j)
#             W[i, j] = 0.1
#         end
#     end
# end 


# open("./graph.tmp.dot", "w") do file
#     write(file, "digraph {\n")

#     for n in vertices(G)  # add nodes
#         write(file,
#               "    $n;\n")
#     end

#     for e in edges(G)  # add edges
#         i=src(e); j=dst(e)
#         write(file,
#               "    $i -> $j;\n")
#     end

#     write(file, "}")
# end

# GraphViz.load("graph.tmp.dot")
# current_path = pwd()
# print(current_path)

# import Base
# Base.run(`dot -V`)
# Base.run(`cd $current_path`)
# Base.run(pipeline(`dot -Tpdf graph.tmp.dot`,
#                stdout="graph.tmp.pdf"))
# Base.run(`open graph.tmp.pdf`)


In [ ]:
import Base
Base.run(`dot -V`)

In [ ]:
import Base

Base.run(`ls`)

In [ ]:
# Represent graphically the model 
# the use of graphviz is better !!!
import Pkg; Pkg.add("GraphViz")
using GraphViz
# I really like graphviz, try with this library
open("./graph.tmp.dot", "w") do file
    write(file, "digraph {\n")

    for n in vertices(g)  # add nodes
        write(file,
              "    $n;\n")
    end

    for e in edges(g)  # add edges
        i=src(e); j=dst(e)
        write(file,
              "    $i -> $j;\n")
    end

    write(file, "}")
end
run(pipeline(`dot -Tpdf ./graph.tmp.dot`,
             stdout="./graph.tmp.pdf"))
run(`xdg-open $filename.pdf`, wait=false)

dot"""
graph graphname {
     // The label attribute can be used to change the label of a node
     a [label="Sub"];
     // Here, the node shape is changed.
     b [shape=house][color=grey];
     // These edges both have different line properties
     a -- b -- c [color=grey];
     b -- d [style=dotted];
 }
"""